In [1]:
import data
import models
import time
from tensorflow.keras.callbacks import CSVLogger, TerminateOnNaN

Using TensorFlow backend.


In [2]:
d = data.Data()

In [3]:
d.split_train_test(28)

In [4]:
d.standarize_data()

In [5]:
vocab_size, max_len, enc_names = d.encode_names()

In [6]:
train_x, train_y, test_x, test_y = d.retreive_data()

In [7]:
multi_train_y = d.make_multi_output_data(train_y)
multi_test_y = d.make_multi_output_data(test_y)

In [8]:
print("Data structure")
print(f"Number of countries: {len(d.countries)}")
# Check if all countries have the same data shape.
for c in d.countries:
    if c.data.shape != d.countries[0].data.shape:
        print(f"{c.name} has shape of {c.data.shape} instead of {d.countries[0].data.shape}")

Data structure
Number of countries: 188


In [9]:
print(f"Shape for country data: {d.countries[0].data.shape}")
print(f"train_x: {train_x.shape}\ntrain_y: {train_y.shape}\ntest_x: {test_x.shape}\ntest_y: {test_y.shape}")
print(f"Shape for multi train y: {multi_train_y.shape} and multi test y: {multi_test_y.shape}")
print(f"Vocab size: {vocab_size} and Max length: {max_len}")
print(f"Enc names: {enc_names.shape} and type: {type(enc_names)}")

Shape for country data: (245, 5)
train_x: (188, 189, 3)
train_y: (188, 28, 3)
test_x: (188, 189, 3)
test_y: (188, 28, 3)
Shape for multi train y: (3, 188, 28) and multi test y: (3, 188, 28)
Vocab size: 235.0 and Max length: 5
Enc names: (188, 5) and type: <class 'numpy.ndarray'>


In [10]:
temporal_shape = train_x[0].shape
word_shape = enc_names[0].shape
units = 100
output_size = 28

Create models.

In [11]:
multi_out_lstm = models.LSTMMultiOutput(temporal_shape, word_shape, units, output_size, activation='tanh')
multi_out_gru = models.GRUMultiOutput(temporal_shape, word_shape, units, output_size)

In [12]:
multi_out_lstm_V2 = models.LSTMMultiOutput_V2(temporal_shape, word_shape, units, output_size, activation='tanh')
multi_out_gru_V2 = models.GRUMultiOutput_V2(temporal_shape, word_shape, units, output_size)

In [13]:
single_out_lstm = models.LSTMSingleOutput(temporal_shape, word_shape, units, output_size, activation='tanh')
single_out_gru = models.GRUSingleOutput(temporal_shape, word_shape, units, output_size)

In [14]:
multi_lstm_quant = models.LSTMMultiOutputQuantile(temporal_shape, word_shape, units, output_size, activation='tanh')
multi_gru_quant = models.GRUMultiOutputQuantile(temporal_shape, word_shape, units, output_size)

In [15]:
single_lstm_quant = models.LSTMSingleOutputQuantile(temporal_shape, word_shape, units, output_size, activation='tanh')
single_gru_quant = models.GRUSingleOutputQuantile(temporal_shape, word_shape, units, output_size)

Print model architecture.

In [16]:
print(multi_out_lstm.summary())
print(multi_out_gru.summary())
print(multi_out_lstm_V2.summary())
print(multi_out_gru_V2.summary())
print(single_out_lstm.summary())
print(single_out_gru.summary())
print(multi_lstm_quant.summary())
print(multi_gru_quant.summary())
print(single_lstm_quant.summary())
print(single_gru_quant.summary())

Model: "LSTMMultiOutput"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
time_series_input (InputLayer)  [(None, 189, 3)]     0                                            
__________________________________________________________________________________________________
country_name_input (InputLayer) [(None, 5)]          0                                            
__________________________________________________________________________________________________
LSTM_encoder (LSTM)             (None, 100)          41600       time_series_input[0][0]          
__________________________________________________________________________________________________
country_name (Dense)            (None, 1)            6           country_name_input[0][0]         
____________________________________________________________________________________

Create logger callbacks.

In [17]:
multi_out_lstm_logger = CSVLogger('multi_out_lstm.csv', separator=',')
multi_out_gru_logger = CSVLogger('multi_out_gru.csv', separator=',')
multi_out_lstm_V2_logger = CSVLogger('multi_out_lstm_V2.csv', separator=',')
multi_out_gru_V2_logger = CSVLogger('multi_out_gru_V2.csv', separator=',')
single_out_lstm_logger = CSVLogger('single_out_lstm.csv', separator=',')
single_out_gru_logger = CSVLogger('single_out_gru.csv', separator=',')

In [18]:
ton_back = TerminateOnNaN()

In [19]:
epochs = 300
verbose=1

Train models.

In [ ]:
multi_out_lstm_hist = multi_out_lstm.fit([train_x, enc_names], [multi_train_y[0], multi_train_y[1], multi_train_y[2]],
                                         epochs=epochs, verbose=verbose, callbacks=[multi_out_lstm_logger, ton_back])

In [ ]:
multi_out_gru_hist = multi_out_gru.fit([train_x, enc_names], [multi_train_y[0], multi_train_y[1], multi_train_y[2]],
                                       epochs=epochs, verbose=verbose, callbacks=[multi_out_gru_logger, ton_back])

In [ ]:
multi_out_lstm_V2_hist = multi_out_lstm_V2.fit([train_x, enc_names], [multi_train_y[0], multi_train_y[1], multi_train_y[2]],
                                               epochs=epochs, verbose=verbose, callbacks=[multi_out_lstm_V2_logger,
                                                                                          ton_back])

In [ ]:
multi_out_gru_V2_hist = multi_out_gru_V2.fit([train_x, enc_names], [multi_train_y[0], multi_train_y[1], multi_train_y[2]],
                                             epochs=epochs, verbose=verbose, callbacks=[multi_out_gru_V2_logger, ton_back])

In [ ]:
single_out_lstm_hist = single_out_lstm.fit([train_x, enc_names], train_y,
                                           epochs=epochs, verbose=verbose, callbacks=[single_out_lstm_logger, ton_back])

In [ ]:
single_out_gru_hist = single_out_gru.fit([train_x, enc_names], train_y,
                                         epochs=epochs, verbose=verbose, callbacks=[single_out_gru_logger, ton_back])

In [20]:
y_multi = {"confirmed_q1": multi_train_y[0],
     "confirmed_q2": multi_train_y[0],
     "confirmed_q3": multi_train_y[0],
     "deceased_q1": multi_train_y[1],
     "deceased_q2": multi_train_y[1],
     "deceased_q3": multi_train_y[1],
     "recovered_q1": multi_train_y[2],
     "recovered_q2": multi_train_y[2],
     "recovered_q3": multi_train_y[2]}

In [21]:
multi_lstm_hist = multi_lstm_quant.fit([train_x, enc_names], y=y_multi, epochs=epochs, verbose=verbose, callbacks=[ton_back])

Epoch 1/300
6/6 [==============================] - 1s 139ms/step - loss: 51.5030 - confirmed_q1_loss: 4.4413 - confirmed_q2_loss: 5.9350 - confirmed_q3_loss: 6.6153 - deceased_q1_loss: 5.7457 - deceased_q2_loss: 5.5977 - deceased_q3_loss: 8.4415 - recovered_q1_loss: 4.3069 - recovered_q2_loss: 5.2206 - recovered_q3_loss: 5.1991 - confirmed_q1_mean_squared_error: 144.9100 - confirmed_q1_root_mean_squared_error: 12.0379 - confirmed_q1_q0.05: 4.4371 - confirmed_q1_q0.5: 4.3619 - confirmed_q1_q0.95: 4.2867 - confirmed_q2_mean_squared_error: 238.4274 - confirmed_q2_root_mean_squared_error: 15.4411 - confirmed_q2_q0.05: 4.7445 - confirmed_q2_q0.5: 5.9312 - confirmed_q2_q0.95: 7.1179 - confirmed_q3_mean_squared_error: 273.0516 - confirmed_q3_root_mean_squared_error: 16.5243 - confirmed_q3_q0.05: 5.8702 - confirmed_q3_q0.5: 6.2407 - confirmed_q3_q0.95: 6.6112 - deceased_q1_mean_squared_error: 208.1114 - deceased_q1_root_mean_squared_error: 14.4261 - deceased_q1_q0.05: 5.7418 - deceased_q1_q0.5

6/6 [==============================] - 1s 116ms/step - loss: 31.7587 - confirmed_q1_loss: 2.3229 - confirmed_q2_loss: 3.8888 - confirmed_q3_loss: 4.4098 - deceased_q1_loss: 3.7059 - deceased_q2_loss: 3.5017 - deceased_q3_loss: 5.2497 - recovered_q1_loss: 2.5273 - recovered_q2_loss: 3.3077 - recovered_q3_loss: 2.8448 - confirmed_q1_mean_squared_error: 70.0921 - confirmed_q1_root_mean_squared_error: 8.3721 - confirmed_q1_q0.05: 2.3157 - confirmed_q1_q0.5: 2.7120 - confirmed_q1_q0.95: 3.1082 - confirmed_q2_mean_squared_error: 123.8178 - confirmed_q2_root_mean_squared_error: 11.1273 - confirmed_q2_q0.05: 2.8746 - confirmed_q2_q0.5: 3.8800 - confirmed_q2_q0.95: 4.8854 - confirmed_q3_mean_squared_error: 149.8268 - confirmed_q3_root_mean_squared_error: 12.2404 - confirmed_q3_q0.05: 4.1229 - confirmed_q3_q0.5: 4.2616 - confirmed_q3_q0.95: 4.4004 - deceased_q1_mean_squared_error: 109.5106 - deceased_q1_root_mean_squared_error: 10.4647 - deceased_q1_q0.05: 3.6967 - deceased_q1_q0.5: 3.6651 - dec

6/6 [==============================] - 1s 133ms/step - loss: 16.9997 - confirmed_q1_loss: 1.0436 - confirmed_q2_loss: 2.2436 - confirmed_q3_loss: 2.4733 - deceased_q1_loss: 1.9295 - deceased_q2_loss: 2.1760 - deceased_q3_loss: 2.7091 - recovered_q1_loss: 1.3158 - recovered_q2_loss: 1.9212 - recovered_q3_loss: 1.1875 - confirmed_q1_mean_squared_error: 35.8808 - confirmed_q1_root_mean_squared_error: 5.9901 - confirmed_q1_q0.05: 1.0423 - confirmed_q1_q0.5: 2.0015 - confirmed_q1_q0.95: 2.9608 - confirmed_q2_mean_squared_error: 48.1912 - confirmed_q2_root_mean_squared_error: 6.9420 - confirmed_q2_q0.05: 1.5949 - confirmed_q2_q0.5: 2.2414 - confirmed_q2_q0.95: 2.8880 - confirmed_q3_mean_squared_error: 78.0423 - confirmed_q3_root_mean_squared_error: 8.8342 - confirmed_q3_q0.05: 3.4584 - confirmed_q3_q0.5: 2.9651 - confirmed_q3_q0.95: 2.4718 - deceased_q1_mean_squared_error: 53.6080 - deceased_q1_root_mean_squared_error: 7.3217 - deceased_q1_q0.05: 1.9278 - deceased_q1_q0.5: 2.4685 - deceased_

6/6 [==============================] - 1s 122ms/step - loss: 9.2514 - confirmed_q1_loss: 0.5206 - confirmed_q2_loss: 1.4237 - confirmed_q3_loss: 1.1946 - deceased_q1_loss: 0.9343 - deceased_q2_loss: 1.4414 - deceased_q3_loss: 1.3256 - recovered_q1_loss: 0.7016 - recovered_q2_loss: 1.2579 - recovered_q3_loss: 0.4517 - confirmed_q1_mean_squared_error: 21.3897 - confirmed_q1_root_mean_squared_error: 4.6249 - confirmed_q1_q0.05: 0.5195 - confirmed_q1_q0.5: 1.5865 - confirmed_q1_q0.95: 2.6536 - confirmed_q2_mean_squared_error: 19.2081 - confirmed_q2_root_mean_squared_error: 4.3827 - confirmed_q2_q0.05: 1.1380 - confirmed_q2_q0.5: 1.4244 - confirmed_q2_q0.95: 1.7108 - confirmed_q3_mean_squared_error: 44.0730 - confirmed_q3_root_mean_squared_error: 6.6388 - confirmed_q3_q0.05: 3.1577 - confirmed_q3_q0.5: 2.1745 - confirmed_q3_q0.95: 1.1913 - deceased_q1_mean_squared_error: 28.4189 - deceased_q1_root_mean_squared_error: 5.3309 - deceased_q1_q0.05: 0.9320 - deceased_q1_q0.5: 1.8449 - deceased_q

6/6 [==============================] - 1s 119ms/step - loss: 5.4557 - confirmed_q1_loss: 0.2665 - confirmed_q2_loss: 1.0712 - confirmed_q3_loss: 0.4960 - deceased_q1_loss: 0.4285 - deceased_q2_loss: 1.0756 - deceased_q3_loss: 0.6058 - recovered_q1_loss: 0.3440 - recovered_q2_loss: 0.9559 - recovered_q3_loss: 0.2122 - confirmed_q1_mean_squared_error: 13.4145 - confirmed_q1_root_mean_squared_error: 3.6626 - confirmed_q1_q0.05: 0.2663 - confirmed_q1_q0.5: 1.2433 - confirmed_q1_q0.95: 2.2203 - confirmed_q2_mean_squared_error: 10.5191 - confirmed_q2_root_mean_squared_error: 3.2433 - confirmed_q2_q0.05: 1.0836 - confirmed_q2_q0.5: 1.0738 - confirmed_q2_q0.95: 1.0639 - confirmed_q3_mean_squared_error: 27.9669 - confirmed_q3_root_mean_squared_error: 5.2884 - confirmed_q3_q0.05: 2.9391 - confirmed_q3_q0.5: 1.7173 - confirmed_q3_q0.95: 0.4954 - deceased_q1_mean_squared_error: 17.5055 - deceased_q1_root_mean_squared_error: 4.1840 - deceased_q1_q0.05: 0.4283 - deceased_q1_q0.5: 1.4948 - deceased_q

...

6/6 [==============================] - 1s 116ms/step - loss: 0.5870 - confirmed_q1_loss: 0.0299 - confirmed_q2_loss: 0.0890 - confirmed_q3_loss: 0.0295 - deceased_q1_loss: 0.0450 - deceased_q2_loss: 0.1462 - deceased_q3_loss: 0.0454 - recovered_q1_loss: 0.0391 - recovered_q2_loss: 0.1217 - recovered_q3_loss: 0.0413 - confirmed_q1_mean_squared_error: 0.3199 - confirmed_q1_root_mean_squared_error: 0.5656 - confirmed_q1_q0.05: 0.0299 - confirmed_q1_q0.5: 0.2259 - confirmed_q1_q0.95: 0.4218 - confirmed_q2_mean_squared_error: 0.0866 - confirmed_q2_root_mean_squared_error: 0.2943 - confirmed_q2_q0.05: 0.0926 - confirmed_q2_q0.5: 0.0890 - confirmed_q2_q0.95: 0.0854 - confirmed_q3_mean_squared_error: 0.2958 - confirmed_q3_root_mean_squared_error: 0.5439 - confirmed_q3_q0.05: 0.4138 - confirmed_q3_q0.5: 0.2216 - confirmed_q3_q0.95: 0.0295 - deceased_q1_mean_squared_error: 0.7755 - deceased_q1_root_mean_squared_error: 0.8806 - deceased_q1_q0.05: 0.0449 - deceased_q1_q0.5: 0.3541 - deceased_q1_q0

6/6 [==============================] - 1s 116ms/step - loss: 0.5857 - confirmed_q1_loss: 0.0297 - confirmed_q2_loss: 0.0896 - confirmed_q3_loss: 0.0289 - deceased_q1_loss: 0.0444 - deceased_q2_loss: 0.1461 - deceased_q3_loss: 0.0451 - recovered_q1_loss: 0.0389 - recovered_q2_loss: 0.1223 - recovered_q3_loss: 0.0408 - confirmed_q1_mean_squared_error: 0.3236 - confirmed_q1_root_mean_squared_error: 0.5688 - confirmed_q1_q0.05: 0.0297 - confirmed_q1_q0.5: 0.2264 - confirmed_q1_q0.95: 0.4230 - confirmed_q2_mean_squared_error: 0.0875 - confirmed_q2_root_mean_squared_error: 0.2958 - confirmed_q2_q0.05: 0.0891 - confirmed_q2_q0.5: 0.0894 - confirmed_q2_q0.95: 0.0897 - confirmed_q3_mean_squared_error: 0.2856 - confirmed_q3_root_mean_squared_error: 0.5345 - confirmed_q3_q0.05: 0.3967 - confirmed_q3_q0.5: 0.2128 - confirmed_q3_q0.95: 0.0289 - deceased_q1_mean_squared_error: 0.7433 - deceased_q1_root_mean_squared_error: 0.8621 - deceased_q1_q0.05: 0.0445 - deceased_q1_q0.5: 0.3431 - deceased_q1_q0

6/6 [==============================] - 1s 115ms/step - loss: 0.5812 - confirmed_q1_loss: 0.0294 - confirmed_q2_loss: 0.0886 - confirmed_q3_loss: 0.0286 - deceased_q1_loss: 0.0448 - deceased_q2_loss: 0.1439 - deceased_q3_loss: 0.0451 - recovered_q1_loss: 0.0386 - recovered_q2_loss: 0.1215 - recovered_q3_loss: 0.0409 - confirmed_q1_mean_squared_error: 0.3133 - confirmed_q1_root_mean_squared_error: 0.5597 - confirmed_q1_q0.05: 0.0293 - confirmed_q1_q0.5: 0.2229 - confirmed_q1_q0.95: 0.4165 - confirmed_q2_mean_squared_error: 0.0864 - confirmed_q2_root_mean_squared_error: 0.2939 - confirmed_q2_q0.05: 0.0891 - confirmed_q2_q0.5: 0.0883 - confirmed_q2_q0.95: 0.0876 - confirmed_q3_mean_squared_error: 0.2765 - confirmed_q3_root_mean_squared_error: 0.5258 - confirmed_q3_q0.05: 0.3870 - confirmed_q3_q0.5: 0.2078 - confirmed_q3_q0.95: 0.0285 - deceased_q1_mean_squared_error: 0.7780 - deceased_q1_root_mean_squared_error: 0.8820 - deceased_q1_q0.05: 0.0447 - deceased_q1_q0.5: 0.3557 - deceased_q1_q0

6/6 [==============================] - 1s 119ms/step - loss: 0.5799 - confirmed_q1_loss: 0.0292 - confirmed_q2_loss: 0.0893 - confirmed_q3_loss: 0.0286 - deceased_q1_loss: 0.0443 - deceased_q2_loss: 0.1411 - deceased_q3_loss: 0.0450 - recovered_q1_loss: 0.0385 - recovered_q2_loss: 0.1233 - recovered_q3_loss: 0.0406 - confirmed_q1_mean_squared_error: 0.3303 - confirmed_q1_root_mean_squared_error: 0.5747 - confirmed_q1_q0.05: 0.0292 - confirmed_q1_q0.5: 0.2282 - confirmed_q1_q0.95: 0.4271 - confirmed_q2_mean_squared_error: 0.0874 - confirmed_q2_root_mean_squared_error: 0.2957 - confirmed_q2_q0.05: 0.0886 - confirmed_q2_q0.5: 0.0895 - confirmed_q2_q0.95: 0.0904 - confirmed_q3_mean_squared_error: 0.2887 - confirmed_q3_root_mean_squared_error: 0.5373 - confirmed_q3_q0.05: 0.4009 - confirmed_q3_q0.5: 0.2148 - confirmed_q3_q0.95: 0.0287 - deceased_q1_mean_squared_error: 0.7502 - deceased_q1_root_mean_squared_error: 0.8661 - deceased_q1_q0.05: 0.0443 - deceased_q1_q0.5: 0.3477 - deceased_q1_q0

6/6 [==============================] - 1s 114ms/step - loss: 0.5802 - confirmed_q1_loss: 0.0296 - confirmed_q2_loss: 0.0919 - confirmed_q3_loss: 0.0286 - deceased_q1_loss: 0.0446 - deceased_q2_loss: 0.1395 - deceased_q3_loss: 0.0451 - recovered_q1_loss: 0.0385 - recovered_q2_loss: 0.1217 - recovered_q3_loss: 0.0406 - confirmed_q1_mean_squared_error: 0.3347 - confirmed_q1_root_mean_squared_error: 0.5785 - confirmed_q1_q0.05: 0.0297 - confirmed_q1_q0.5: 0.2302 - confirmed_q1_q0.95: 0.4307 - confirmed_q2_mean_squared_error: 0.0884 - confirmed_q2_root_mean_squared_error: 0.2972 - confirmed_q2_q0.05: 0.0935 - confirmed_q2_q0.5: 0.0921 - confirmed_q2_q0.95: 0.0908 - confirmed_q3_mean_squared_error: 0.2995 - confirmed_q3_root_mean_squared_error: 0.5473 - confirmed_q3_q0.05: 0.4078 - confirmed_q3_q0.5: 0.2182 - confirmed_q3_q0.95: 0.0286 - deceased_q1_mean_squared_error: 0.7459 - deceased_q1_root_mean_squared_error: 0.8637 - deceased_q1_q0.05: 0.0446 - deceased_q1_q0.5: 0.3432 - deceased_q1_q0

In [22]:
multi_gru_hist = multi_gru_quant.fit([train_x, enc_names], y=y_multi, epochs=epochs, verbose=verbose, callbacks=[ton_back])

Epoch 1/300
6/6 [==============================] - 1s 194ms/step - loss: 31.9489 - confirmed_q1_loss: 3.6004 - confirmed_q2_loss: 4.1141 - confirmed_q3_loss: 3.4269 - deceased_q1_loss: 4.6098 - deceased_q2_loss: 3.9259 - deceased_q3_loss: 2.7900 - recovered_q1_loss: 2.3801 - recovered_q2_loss: 3.8590 - recovered_q3_loss: 3.2426 - confirmed_q1_mean_squared_error: 87.1672 - confirmed_q1_root_mean_squared_error: 9.3363 - confirmed_q1_q0.05: 3.6004 - confirmed_q1_q0.5: 3.4360 - confirmed_q1_q0.95: 3.2715 - confirmed_q2_mean_squared_error: 116.4508 - confirmed_q2_root_mean_squared_error: 10.7912 - confirmed_q2_q0.05: 4.6278 - confirmed_q2_q0.5: 4.1156 - confirmed_q2_q0.95: 3.6035 - confirmed_q3_mean_squared_error: 67.6877 - confirmed_q3_root_mean_squared_error: 8.2273 - confirmed_q3_q0.05: 2.4471 - confirmed_q3_q0.5: 2.9373 - confirmed_q3_q0.95: 3.4275 - deceased_q1_mean_squared_error: 101.8840 - deceased_q1_root_mean_squared_error: 10.0938 - deceased_q1_q0.05: 4.6110 - deceased_q1_q0.5: 3.

6/6 [==============================] - 1s 172ms/step - loss: 17.5163 - confirmed_q1_loss: 1.9225 - confirmed_q2_loss: 2.6164 - confirmed_q3_loss: 1.5597 - deceased_q1_loss: 2.4515 - deceased_q2_loss: 2.5336 - deceased_q3_loss: 1.1345 - recovered_q1_loss: 1.2904 - recovered_q2_loss: 2.5897 - recovered_q3_loss: 1.4181 - confirmed_q1_mean_squared_error: 58.1163 - confirmed_q1_root_mean_squared_error: 7.6234 - confirmed_q1_q0.05: 1.9134 - confirmed_q1_q0.5: 2.7013 - confirmed_q1_q0.95: 3.4893 - confirmed_q2_mean_squared_error: 54.4983 - confirmed_q2_root_mean_squared_error: 7.3823 - confirmed_q2_q0.05: 2.8085 - confirmed_q2_q0.5: 2.6077 - confirmed_q2_q0.95: 2.4069 - confirmed_q3_mean_squared_error: 36.5585 - confirmed_q3_root_mean_squared_error: 6.0464 - confirmed_q3_q0.05: 2.6898 - confirmed_q3_q0.5: 2.1216 - confirmed_q3_q0.95: 1.5534 - deceased_q1_mean_squared_error: 57.8721 - deceased_q1_root_mean_squared_error: 7.6074 - deceased_q1_q0.05: 2.4418 - deceased_q1_q0.5: 2.8883 - deceased_

6/6 [==============================] - 1s 191ms/step - loss: 7.6975 - confirmed_q1_loss: 0.4467 - confirmed_q2_loss: 1.7204 - confirmed_q3_loss: 0.3792 - deceased_q1_loss: 0.6079 - deceased_q2_loss: 1.6611 - deceased_q3_loss: 0.4248 - recovered_q1_loss: 0.3822 - recovered_q2_loss: 1.6168 - recovered_q3_loss: 0.4584 - confirmed_q1_mean_squared_error: 59.5949 - confirmed_q1_root_mean_squared_error: 7.7198 - confirmed_q1_q0.05: 0.4445 - confirmed_q1_q0.5: 3.0782 - confirmed_q1_q0.95: 5.7120 - confirmed_q2_mean_squared_error: 25.5275 - confirmed_q2_root_mean_squared_error: 5.0525 - confirmed_q2_q0.05: 1.5144 - confirmed_q2_q0.5: 1.7099 - confirmed_q2_q0.95: 1.9055 - confirmed_q3_mean_squared_error: 50.0063 - confirmed_q3_root_mean_squared_error: 7.0715 - confirmed_q3_q0.05: 5.0799 - confirmed_q3_q0.5: 2.7286 - confirmed_q3_q0.95: 0.3773 - deceased_q1_mean_squared_error: 79.0001 - deceased_q1_root_mean_squared_error: 8.8882 - deceased_q1_q0.05: 0.6052 - deceased_q1_q0.5: 3.6231 - deceased_q

6/6 [==============================] - 1s 191ms/step - loss: 5.2522 - confirmed_q1_loss: 0.2594 - confirmed_q2_loss: 1.3084 - confirmed_q3_loss: 0.2053 - deceased_q1_loss: 0.3015 - deceased_q2_loss: 1.2272 - deceased_q3_loss: 0.2531 - recovered_q1_loss: 0.2195 - recovered_q2_loss: 1.1974 - recovered_q3_loss: 0.2805 - confirmed_q1_mean_squared_error: 35.7291 - confirmed_q1_root_mean_squared_error: 5.9774 - confirmed_q1_q0.05: 0.2585 - confirmed_q1_q0.5: 2.2375 - confirmed_q1_q0.95: 4.2166 - confirmed_q2_mean_squared_error: 14.8269 - confirmed_q2_root_mean_squared_error: 3.8506 - confirmed_q2_q0.05: 1.2544 - confirmed_q2_q0.5: 1.3033 - confirmed_q2_q0.95: 1.3521 - confirmed_q3_mean_squared_error: 27.2519 - confirmed_q3_root_mean_squared_error: 5.2203 - confirmed_q3_q0.05: 3.3394 - confirmed_q3_q0.5: 1.7721 - confirmed_q3_q0.95: 0.2047 - deceased_q1_mean_squared_error: 42.0740 - deceased_q1_root_mean_squared_error: 6.4864 - deceased_q1_q0.05: 0.3004 - deceased_q1_q0.5: 2.4996 - deceased_q

6/6 [==============================] - 1s 142ms/step - loss: 4.1508 - confirmed_q1_loss: 0.1926 - confirmed_q2_loss: 1.0521 - confirmed_q3_loss: 0.1565 - deceased_q1_loss: 0.2383 - deceased_q2_loss: 0.9772 - deceased_q3_loss: 0.1836 - recovered_q1_loss: 0.1675 - recovered_q2_loss: 0.9618 - recovered_q3_loss: 0.2212 - confirmed_q1_mean_squared_error: 22.5763 - confirmed_q1_root_mean_squared_error: 4.7515 - confirmed_q1_q0.05: 0.1945 - confirmed_q1_q0.5: 1.5620 - confirmed_q1_q0.95: 2.9295 - confirmed_q2_mean_squared_error: 10.3341 - confirmed_q2_root_mean_squared_error: 3.2147 - confirmed_q2_q0.05: 1.0233 - confirmed_q2_q0.5: 1.0545 - confirmed_q2_q0.95: 1.0857 - confirmed_q3_mean_squared_error: 17.0377 - confirmed_q3_root_mean_squared_error: 4.1277 - confirmed_q3_q0.05: 2.2672 - confirmed_q3_q0.5: 1.2127 - confirmed_q3_q0.95: 0.1583 - deceased_q1_mean_squared_error: 27.3507 - deceased_q1_root_mean_squared_error: 5.2298 - deceased_q1_q0.05: 0.2408 - deceased_q1_q0.5: 1.8896 - deceased_q

...

6/6 [==============================] - 1s 153ms/step - loss: 0.4728 - confirmed_q1_loss: 0.0219 - confirmed_q2_loss: 0.0760 - confirmed_q3_loss: 0.0217 - deceased_q1_loss: 0.0363 - deceased_q2_loss: 0.1128 - deceased_q3_loss: 0.0366 - recovered_q1_loss: 0.0318 - recovered_q2_loss: 0.1034 - recovered_q3_loss: 0.0323 - confirmed_q1_mean_squared_error: 0.2212 - confirmed_q1_root_mean_squared_error: 0.4703 - confirmed_q1_q0.05: 0.0221 - confirmed_q1_q0.5: 0.1744 - confirmed_q1_q0.95: 0.3267 - confirmed_q2_mean_squared_error: 0.0628 - confirmed_q2_root_mean_squared_error: 0.2507 - confirmed_q2_q0.05: 0.0793 - confirmed_q2_q0.5: 0.0765 - confirmed_q2_q0.95: 0.0736 - confirmed_q3_mean_squared_error: 0.2007 - confirmed_q3_root_mean_squared_error: 0.4480 - confirmed_q3_q0.05: 0.3076 - confirmed_q3_q0.5: 0.1647 - confirmed_q3_q0.95: 0.0218 - deceased_q1_mean_squared_error: 0.5065 - deceased_q1_root_mean_squared_error: 0.7117 - deceased_q1_q0.05: 0.0363 - deceased_q1_q0.5: 0.2758 - deceased_q1_q0

6/6 [==============================] - 1s 141ms/step - loss: 0.4722 - confirmed_q1_loss: 0.0217 - confirmed_q2_loss: 0.0759 - confirmed_q3_loss: 0.0214 - deceased_q1_loss: 0.0353 - deceased_q2_loss: 0.1124 - deceased_q3_loss: 0.0361 - recovered_q1_loss: 0.0321 - recovered_q2_loss: 0.1047 - recovered_q3_loss: 0.0326 - confirmed_q1_mean_squared_error: 0.2331 - confirmed_q1_root_mean_squared_error: 0.4828 - confirmed_q1_q0.05: 0.0218 - confirmed_q1_q0.5: 0.1764 - confirmed_q1_q0.95: 0.3310 - confirmed_q2_mean_squared_error: 0.0637 - confirmed_q2_root_mean_squared_error: 0.2524 - confirmed_q2_q0.05: 0.0772 - confirmed_q2_q0.5: 0.0759 - confirmed_q2_q0.95: 0.0747 - confirmed_q3_mean_squared_error: 0.2288 - confirmed_q3_root_mean_squared_error: 0.4783 - confirmed_q3_q0.05: 0.3377 - confirmed_q3_q0.5: 0.1796 - confirmed_q3_q0.95: 0.0214 - deceased_q1_mean_squared_error: 0.5016 - deceased_q1_root_mean_squared_error: 0.7083 - deceased_q1_q0.05: 0.0352 - deceased_q1_q0.5: 0.2697 - deceased_q1_q0

6/6 [==============================] - 1s 136ms/step - loss: 0.4651 - confirmed_q1_loss: 0.0215 - confirmed_q2_loss: 0.0759 - confirmed_q3_loss: 0.0213 - deceased_q1_loss: 0.0352 - deceased_q2_loss: 0.1092 - deceased_q3_loss: 0.0356 - recovered_q1_loss: 0.0317 - recovered_q2_loss: 0.1022 - recovered_q3_loss: 0.0323 - confirmed_q1_mean_squared_error: 0.2245 - confirmed_q1_root_mean_squared_error: 0.4739 - confirmed_q1_q0.05: 0.0215 - confirmed_q1_q0.5: 0.1708 - confirmed_q1_q0.95: 0.3201 - confirmed_q2_mean_squared_error: 0.0635 - confirmed_q2_root_mean_squared_error: 0.2520 - confirmed_q2_q0.05: 0.0783 - confirmed_q2_q0.5: 0.0759 - confirmed_q2_q0.95: 0.0734 - confirmed_q3_mean_squared_error: 0.2113 - confirmed_q3_root_mean_squared_error: 0.4597 - confirmed_q3_q0.05: 0.3127 - confirmed_q3_q0.5: 0.1670 - confirmed_q3_q0.95: 0.0214 - deceased_q1_mean_squared_error: 0.4846 - deceased_q1_root_mean_squared_error: 0.6961 - deceased_q1_q0.05: 0.0351 - deceased_q1_q0.5: 0.2618 - deceased_q1_q0

6/6 [==============================] - 1s 133ms/step - loss: 0.4655 - confirmed_q1_loss: 0.0219 - confirmed_q2_loss: 0.0753 - confirmed_q3_loss: 0.0215 - deceased_q1_loss: 0.0355 - deceased_q2_loss: 0.1101 - deceased_q3_loss: 0.0358 - recovered_q1_loss: 0.0314 - recovered_q2_loss: 0.1019 - recovered_q3_loss: 0.0320 - confirmed_q1_mean_squared_error: 0.2390 - confirmed_q1_root_mean_squared_error: 0.4889 - confirmed_q1_q0.05: 0.0218 - confirmed_q1_q0.5: 0.1811 - confirmed_q1_q0.95: 0.3404 - confirmed_q2_mean_squared_error: 0.0629 - confirmed_q2_root_mean_squared_error: 0.2507 - confirmed_q2_q0.05: 0.0779 - confirmed_q2_q0.5: 0.0750 - confirmed_q2_q0.95: 0.0721 - confirmed_q3_mean_squared_error: 0.2050 - confirmed_q3_root_mean_squared_error: 0.4527 - confirmed_q3_q0.05: 0.3123 - confirmed_q3_q0.5: 0.1669 - confirmed_q3_q0.95: 0.0215 - deceased_q1_mean_squared_error: 0.5056 - deceased_q1_root_mean_squared_error: 0.7111 - deceased_q1_q0.05: 0.0354 - deceased_q1_q0.5: 0.2709 - deceased_q1_q0

6/6 [==============================] - 1s 137ms/step - loss: 0.4616 - confirmed_q1_loss: 0.0211 - confirmed_q2_loss: 0.0748 - confirmed_q3_loss: 0.0212 - deceased_q1_loss: 0.0353 - deceased_q2_loss: 0.1092 - deceased_q3_loss: 0.0354 - recovered_q1_loss: 0.0311 - recovered_q2_loss: 0.1019 - recovered_q3_loss: 0.0317 - confirmed_q1_mean_squared_error: 0.2246 - confirmed_q1_root_mean_squared_error: 0.4739 - confirmed_q1_q0.05: 0.0211 - confirmed_q1_q0.5: 0.1726 - confirmed_q1_q0.95: 0.3241 - confirmed_q2_mean_squared_error: 0.0617 - confirmed_q2_root_mean_squared_error: 0.2485 - confirmed_q2_q0.05: 0.0759 - confirmed_q2_q0.5: 0.0746 - confirmed_q2_q0.95: 0.0733 - confirmed_q3_mean_squared_error: 0.2174 - confirmed_q3_root_mean_squared_error: 0.4663 - confirmed_q3_q0.05: 0.3250 - confirmed_q3_q0.5: 0.1731 - confirmed_q3_q0.95: 0.0211 - deceased_q1_mean_squared_error: 0.5041 - deceased_q1_root_mean_squared_error: 0.7100 - deceased_q1_q0.05: 0.0355 - deceased_q1_q0.5: 0.2709 - deceased_q1_q0

In [23]:
y_single = {"output_q1": train_y,
            "output_q2": train_y,
            "output_q3": train_y}

In [24]:
single_lstm_hist = single_lstm_quant.fit([train_x, enc_names], y=y_single, epochs=epochs, verbose=verbose, callbacks=[ton_back])

Epoch 1/300
6/6 [==============================] - 1s 165ms/step - loss: 9.2050 - output_q1_loss: 3.1806 - output_q2_loss: 3.3783 - output_q3_loss: 2.6462 - output_q1_mean_squared_error: 121.1403 - output_q1_root_mean_squared_error: 11.0064 - output_q1_q0.05: 3.1653 - output_q1_q0.5: 4.6742 - output_q1_q0.95: 6.1830 - output_q2_mean_squared_error: 71.7608 - output_q2_root_mean_squared_error: 8.4712 - output_q2_q0.05: 4.1331 - output_q2_q0.5: 3.3639 - output_q2_q0.95: 2.5947 - output_q3_mean_squared_error: 53.5684 - output_q3_root_mean_squared_error: 7.3190 - output_q3_q0.05: 2.9427 - output_q3_q0.5: 2.7860 - output_q3_q0.95: 2.6292
Epoch 2/300
6/6 [==============================] - 1s 142ms/step - loss: 7.4138 - output_q1_loss: 2.6652 - output_q2_loss: 3.0244 - output_q3_loss: 1.7242 - output_q1_mean_squared_error: 100.5289 - output_q1_root_mean_squared_error: 10.0264 - output_q1_q0.05: 2.6663 - output_q1_q0.5: 4.2226 - output_q1_q0.95: 5.7789 - output_q2_mean_squared_error: 60.1872 - 

6/6 [==============================] - 1s 116ms/step - loss: 1.3884 - output_q1_loss: 0.2979 - output_q2_loss: 0.8954 - output_q3_loss: 0.1951 - output_q1_mean_squared_error: 32.6330 - output_q1_root_mean_squared_error: 5.7125 - output_q1_q0.05: 0.2975 - output_q1_q0.5: 2.4524 - output_q1_q0.95: 4.6074 - output_q2_mean_squared_error: 5.9168 - output_q2_root_mean_squared_error: 2.4324 - output_q2_q0.05: 0.8049 - output_q2_q0.5: 0.8953 - output_q2_q0.95: 0.9856 - output_q3_mean_squared_error: 15.1886 - output_q3_root_mean_squared_error: 3.8973 - output_q3_q0.05: 3.0130 - output_q3_q0.5: 1.6039 - output_q3_q0.95: 0.1947
Epoch 14/300
6/6 [==============================] - 1s 114ms/step - loss: 1.3381 - output_q1_loss: 0.2868 - output_q2_loss: 0.8626 - output_q3_loss: 0.1887 - output_q1_mean_squared_error: 30.0883 - output_q1_root_mean_squared_error: 5.4853 - output_q1_q0.05: 0.2875 - output_q1_q0.5: 2.3564 - output_q1_q0.95: 4.4253 - output_q2_mean_squared_error: 5.4934 - output_q2_root_me

6/6 [==============================] - 1s 117ms/step - loss: 0.8679 - output_q1_loss: 0.1827 - output_q2_loss: 0.5481 - output_q3_loss: 0.1371 - output_q1_mean_squared_error: 13.9032 - output_q1_root_mean_squared_error: 3.7287 - output_q1_q0.05: 0.1835 - output_q1_q0.5: 1.4292 - output_q1_q0.95: 2.6749 - output_q2_mean_squared_error: 2.0677 - output_q2_root_mean_squared_error: 1.4380 - output_q2_q0.05: 0.5348 - output_q2_q0.5: 0.5485 - output_q2_q0.95: 0.5621 - output_q3_mean_squared_error: 7.2452 - output_q3_root_mean_squared_error: 2.6917 - output_q3_q0.05: 2.0482 - output_q3_q0.5: 1.0929 - output_q3_q0.95: 0.1376
Epoch 26/300
6/6 [==============================] - 1s 114ms/step - loss: 0.8398 - output_q1_loss: 0.1770 - output_q2_loss: 0.5274 - output_q3_loss: 0.1353 - output_q1_mean_squared_error: 13.3979 - output_q1_root_mean_squared_error: 3.6603 - output_q1_q0.05: 0.1764 - output_q1_q0.5: 1.3856 - output_q1_q0.95: 2.5947 - output_q2_mean_squared_error: 1.9065 - output_q2_root_mea

6/6 [==============================] - 1s 125ms/step - loss: 0.6878 - output_q1_loss: 0.1598 - output_q2_loss: 0.4030 - output_q3_loss: 0.1250 - output_q1_mean_squared_error: 11.3085 - output_q1_root_mean_squared_error: 3.3628 - output_q1_q0.05: 0.1594 - output_q1_q0.5: 1.2666 - output_q1_q0.95: 2.3738 - output_q2_mean_squared_error: 0.9956 - output_q2_root_mean_squared_error: 0.9978 - output_q2_q0.05: 0.3972 - output_q2_q0.5: 0.4030 - output_q2_q0.95: 0.4087 - output_q3_mean_squared_error: 6.0968 - output_q3_root_mean_squared_error: 2.4692 - output_q3_q0.05: 1.9222 - output_q3_q0.5: 1.0235 - output_q3_q0.95: 0.1249
Epoch 38/300
6/6 [==============================] - 1s 114ms/step - loss: 0.6828 - output_q1_loss: 0.1591 - output_q2_loss: 0.3997 - output_q3_loss: 0.1241 - output_q1_mean_squared_error: 11.2526 - output_q1_root_mean_squared_error: 3.3545 - output_q1_q0.05: 0.1585 - output_q1_q0.5: 1.2713 - output_q1_q0.95: 2.3842 - output_q2_mean_squared_error: 0.9750 - output_q2_root_mea

6/6 [==============================] - 1s 116ms/step - loss: 0.6540 - output_q1_loss: 0.1498 - output_q2_loss: 0.3899 - output_q3_loss: 0.1143 - output_q1_mean_squared_error: 9.9315 - output_q1_root_mean_squared_error: 3.1514 - output_q1_q0.05: 0.1497 - output_q1_q0.5: 1.2112 - output_q1_q0.95: 2.2727 - output_q2_mean_squared_error: 0.9144 - output_q2_root_mean_squared_error: 0.9562 - output_q2_q0.05: 0.3790 - output_q2_q0.5: 0.3896 - output_q2_q0.95: 0.4001 - output_q3_mean_squared_error: 5.1612 - output_q3_root_mean_squared_error: 2.2718 - output_q3_q0.05: 1.8146 - output_q3_q0.5: 0.9644 - output_q3_q0.95: 0.1142
Epoch 50/300
6/6 [==============================] - 1s 114ms/step - loss: 0.6517 - output_q1_loss: 0.1489 - output_q2_loss: 0.3894 - output_q3_loss: 0.1133 - output_q1_mean_squared_error: 9.7684 - output_q1_root_mean_squared_error: 3.1255 - output_q1_q0.05: 0.1485 - output_q1_q0.5: 1.1950 - output_q1_q0.95: 2.2416 - output_q2_mean_squared_error: 0.9132 - output_q2_root_mean_

...

6/6 [==============================] - 1s 113ms/step - loss: 0.5619 - output_q1_loss: 0.0852 - output_q2_loss: 0.3882 - output_q3_loss: 0.0885 - output_q1_mean_squared_error: 2.9346 - output_q1_root_mean_squared_error: 1.7131 - output_q1_q0.05: 0.0851 - output_q1_q0.5: 0.7130 - output_q1_q0.95: 1.3410 - output_q2_mean_squared_error: 0.9170 - output_q2_root_mean_squared_error: 0.9576 - output_q2_q0.05: 0.3763 - output_q2_q0.5: 0.3881 - output_q2_q0.95: 0.3999 - output_q3_mean_squared_error: 3.2166 - output_q3_root_mean_squared_error: 1.7935 - output_q3_q0.05: 1.4397 - output_q3_q0.5: 0.7641 - output_q3_q0.95: 0.0885
Epoch 242/300
6/6 [==============================] - 1s 115ms/step - loss: 0.5623 - output_q1_loss: 0.0851 - output_q2_loss: 0.3886 - output_q3_loss: 0.0886 - output_q1_mean_squared_error: 3.1125 - output_q1_root_mean_squared_error: 1.7642 - output_q1_q0.05: 0.0851 - output_q1_q0.5: 0.7422 - output_q1_q0.95: 1.3994 - output_q2_mean_squared_error: 0.9151 - output_q2_root_mean

6/6 [==============================] - 1s 115ms/step - loss: 0.5623 - output_q1_loss: 0.0851 - output_q2_loss: 0.3886 - output_q3_loss: 0.0885 - output_q1_mean_squared_error: 3.0505 - output_q1_root_mean_squared_error: 1.7466 - output_q1_q0.05: 0.0850 - output_q1_q0.5: 0.7304 - output_q1_q0.95: 1.3757 - output_q2_mean_squared_error: 0.9164 - output_q2_root_mean_squared_error: 0.9573 - output_q2_q0.05: 0.3921 - output_q2_q0.5: 0.3884 - output_q2_q0.95: 0.3847 - output_q3_mean_squared_error: 3.1940 - output_q3_root_mean_squared_error: 1.7872 - output_q3_q0.05: 1.4346 - output_q3_q0.5: 0.7616 - output_q3_q0.95: 0.0885
Epoch 254/300
6/6 [==============================] - 1s 128ms/step - loss: 0.5617 - output_q1_loss: 0.0852 - output_q2_loss: 0.3880 - output_q3_loss: 0.0885 - output_q1_mean_squared_error: 3.0747 - output_q1_root_mean_squared_error: 1.7535 - output_q1_q0.05: 0.0852 - output_q1_q0.5: 0.7362 - output_q1_q0.95: 1.3873 - output_q2_mean_squared_error: 0.9169 - output_q2_root_mean

6/6 [==============================] - 1s 116ms/step - loss: 0.5625 - output_q1_loss: 0.0850 - output_q2_loss: 0.3892 - output_q3_loss: 0.0883 - output_q1_mean_squared_error: 3.0531 - output_q1_root_mean_squared_error: 1.7473 - output_q1_q0.05: 0.0850 - output_q1_q0.5: 0.7326 - output_q1_q0.95: 1.3802 - output_q2_mean_squared_error: 0.9163 - output_q2_root_mean_squared_error: 0.9573 - output_q2_q0.05: 0.3790 - output_q2_q0.5: 0.3892 - output_q2_q0.95: 0.3993 - output_q3_mean_squared_error: 3.1299 - output_q3_root_mean_squared_error: 1.7691 - output_q3_q0.05: 1.4143 - output_q3_q0.5: 0.7514 - output_q3_q0.95: 0.0885
Epoch 266/300
6/6 [==============================] - 1s 127ms/step - loss: 0.5624 - output_q1_loss: 0.0849 - output_q2_loss: 0.3889 - output_q3_loss: 0.0885 - output_q1_mean_squared_error: 3.0142 - output_q1_root_mean_squared_error: 1.7361 - output_q1_q0.05: 0.0849 - output_q1_q0.5: 0.7247 - output_q1_q0.95: 1.3646 - output_q2_mean_squared_error: 0.9173 - output_q2_root_mean

6/6 [==============================] - 1s 115ms/step - loss: 0.5621 - output_q1_loss: 0.0850 - output_q2_loss: 0.3886 - output_q3_loss: 0.0885 - output_q1_mean_squared_error: 3.0437 - output_q1_root_mean_squared_error: 1.7446 - output_q1_q0.05: 0.0849 - output_q1_q0.5: 0.7290 - output_q1_q0.95: 1.3731 - output_q2_mean_squared_error: 0.9174 - output_q2_root_mean_squared_error: 0.9578 - output_q2_q0.05: 0.3900 - output_q2_q0.5: 0.3886 - output_q2_q0.95: 0.3871 - output_q3_mean_squared_error: 3.1270 - output_q3_root_mean_squared_error: 1.7683 - output_q3_q0.05: 1.4102 - output_q3_q0.5: 0.7494 - output_q3_q0.95: 0.0885
Epoch 278/300
6/6 [==============================] - 1s 114ms/step - loss: 0.5622 - output_q1_loss: 0.0849 - output_q2_loss: 0.3889 - output_q3_loss: 0.0884 - output_q1_mean_squared_error: 3.0094 - output_q1_root_mean_squared_error: 1.7348 - output_q1_q0.05: 0.0849 - output_q1_q0.5: 0.7247 - output_q1_q0.95: 1.3645 - output_q2_mean_squared_error: 0.9190 - output_q2_root_mean

6/6 [==============================] - 1s 114ms/step - loss: 0.5622 - output_q1_loss: 0.0846 - output_q2_loss: 0.3894 - output_q3_loss: 0.0882 - output_q1_mean_squared_error: 3.0339 - output_q1_root_mean_squared_error: 1.7418 - output_q1_q0.05: 0.0847 - output_q1_q0.5: 0.7273 - output_q1_q0.95: 1.3699 - output_q2_mean_squared_error: 0.9189 - output_q2_root_mean_squared_error: 0.9586 - output_q2_q0.05: 0.3912 - output_q2_q0.5: 0.3894 - output_q2_q0.95: 0.3876 - output_q3_mean_squared_error: 3.1850 - output_q3_root_mean_squared_error: 1.7846 - output_q3_q0.05: 1.4275 - output_q3_q0.5: 0.7578 - output_q3_q0.95: 0.0880
Epoch 290/300
6/6 [==============================] - 1s 115ms/step - loss: 0.5626 - output_q1_loss: 0.0849 - output_q2_loss: 0.3894 - output_q3_loss: 0.0883 - output_q1_mean_squared_error: 3.0886 - output_q1_root_mean_squared_error: 1.7575 - output_q1_q0.05: 0.0849 - output_q1_q0.5: 0.7387 - output_q1_q0.95: 1.3926 - output_q2_mean_squared_error: 0.9176 - output_q2_root_mean

In [25]:
single_gru_hist = single_gru_quant.fit([train_x, enc_names], y=y_single, epochs=epochs, verbose=verbose, callbacks=[ton_back])

Epoch 1/300
6/6 [==============================] - 1s 186ms/step - loss: 25.1412 - output_q1_loss: 7.7575 - output_q2_loss: 6.8198 - output_q3_loss: 10.5639 - output_q1_mean_squared_error: 268.1298 - output_q1_root_mean_squared_error: 16.3747 - output_q1_q0.05: 7.7737 - output_q1_q0.5: 7.0197 - output_q1_q0.95: 6.2658 - output_q2_mean_squared_error: 364.0940 - output_q2_root_mean_squared_error: 19.0812 - output_q2_q0.05: 11.6884 - output_q2_q0.5: 6.8356 - output_q2_q0.95: 1.9827 - output_q3_mean_squared_error: 478.9779 - output_q3_root_mean_squared_error: 21.8856 - output_q3_q0.05: 8.0702 - output_q3_q0.5: 9.3298 - output_q3_q0.95: 10.5894
Epoch 2/300
6/6 [==============================] - 1s 166ms/step - loss: 23.2040 - output_q1_loss: 7.0694 - output_q2_loss: 6.3796 - output_q3_loss: 9.7551 - output_q1_mean_squared_error: 237.8582 - output_q1_root_mean_squared_error: 15.4227 - output_q1_q0.05: 7.0803 - output_q1_q0.5: 6.5443 - output_q1_q0.95: 6.0083 - output_q2_mean_squared_error: 3

6/6 [==============================] - 1s 135ms/step - loss: 4.6111 - output_q1_loss: 0.7511 - output_q2_loss: 2.8204 - output_q3_loss: 1.0396 - output_q1_mean_squared_error: 240.9340 - output_q1_root_mean_squared_error: 15.5220 - output_q1_q0.05: 0.7521 - output_q1_q0.5: 6.7191 - output_q1_q0.95: 12.6862 - output_q2_mean_squared_error: 88.1129 - output_q2_root_mean_squared_error: 9.3868 - output_q2_q0.05: 2.0000 - output_q2_q0.5: 2.8291 - output_q2_q0.95: 3.6581 - output_q3_mean_squared_error: 435.0901 - output_q3_root_mean_squared_error: 20.8588 - output_q3_q0.05: 15.9107 - output_q3_q0.5: 8.4759 - output_q3_q0.95: 1.0410
Epoch 14/300
6/6 [==============================] - 1s 137ms/step - loss: 4.4131 - output_q1_loss: 0.7006 - output_q2_loss: 2.7105 - output_q3_loss: 1.0019 - output_q1_mean_squared_error: 220.3561 - output_q1_root_mean_squared_error: 14.8444 - output_q1_q0.05: 0.6995 - output_q1_q0.5: 6.3677 - output_q1_q0.95: 12.0360 - output_q2_mean_squared_error: 81.9104 - output

6/6 [==============================] - 1s 133ms/step - loss: 2.7359 - output_q1_loss: 0.4293 - output_q2_loss: 1.5582 - output_q3_loss: 0.7483 - output_q1_mean_squared_error: 79.1097 - output_q1_root_mean_squared_error: 8.8944 - output_q1_q0.05: 0.4280 - output_q1_q0.5: 3.6938 - output_q1_q0.95: 6.9597 - output_q2_mean_squared_error: 31.0873 - output_q2_root_mean_squared_error: 5.5756 - output_q2_q0.05: 1.3691 - output_q2_q0.5: 1.5551 - output_q2_q0.95: 1.7411 - output_q3_mean_squared_error: 233.9478 - output_q3_root_mean_squared_error: 15.2954 - output_q3_q0.05: 11.5374 - output_q3_q0.5: 6.1415 - output_q3_q0.95: 0.7457
Epoch 26/300
6/6 [==============================] - 1s 127ms/step - loss: 2.6151 - output_q1_loss: 0.4170 - output_q2_loss: 1.4791 - output_q3_loss: 0.7190 - output_q1_mean_squared_error: 84.3526 - output_q1_root_mean_squared_error: 9.1844 - output_q1_q0.05: 0.4170 - output_q1_q0.5: 3.8747 - output_q1_q0.95: 7.3323 - output_q2_mean_squared_error: 29.2835 - output_q2_ro

6/6 [==============================] - 1s 134ms/step - loss: 1.6089 - output_q1_loss: 0.2594 - output_q2_loss: 0.8593 - output_q3_loss: 0.4903 - output_q1_mean_squared_error: 43.7660 - output_q1_root_mean_squared_error: 6.6156 - output_q1_q0.05: 0.2599 - output_q1_q0.5: 2.3969 - output_q1_q0.95: 4.5339 - output_q2_mean_squared_error: 8.1594 - output_q2_root_mean_squared_error: 2.8565 - output_q2_q0.05: 0.6803 - output_q2_q0.5: 0.8582 - output_q2_q0.95: 1.0361 - output_q3_mean_squared_error: 142.8991 - output_q3_root_mean_squared_error: 11.9540 - output_q3_q0.05: 8.5761 - output_q3_q0.5: 4.5335 - output_q3_q0.95: 0.4908
Epoch 38/300
6/6 [==============================] - 1s 148ms/step - loss: 1.5321 - output_q1_loss: 0.2551 - output_q2_loss: 0.8059 - output_q3_loss: 0.4711 - output_q1_mean_squared_error: 40.5849 - output_q1_root_mean_squared_error: 6.3706 - output_q1_q0.05: 0.2558 - output_q1_q0.5: 2.2978 - output_q1_q0.95: 4.3397 - output_q2_mean_squared_error: 6.6476 - output_q2_root_

6/6 [==============================] - 1s 167ms/step - loss: 0.9711 - output_q1_loss: 0.2184 - output_q2_loss: 0.4027 - output_q3_loss: 0.3500 - output_q1_mean_squared_error: 29.5993 - output_q1_root_mean_squared_error: 5.4405 - output_q1_q0.05: 0.2189 - output_q1_q0.5: 1.9882 - output_q1_q0.95: 3.7574 - output_q2_mean_squared_error: 0.9944 - output_q2_root_mean_squared_error: 0.9972 - output_q2_q0.05: 0.4252 - output_q2_q0.5: 0.4031 - output_q2_q0.95: 0.3810 - output_q3_mean_squared_error: 97.9735 - output_q3_root_mean_squared_error: 9.8982 - output_q3_q0.05: 6.1522 - output_q3_q0.5: 3.2515 - output_q3_q0.95: 0.3509
Epoch 50/300
6/6 [==============================] - 1s 150ms/step - loss: 0.9569 - output_q1_loss: 0.2148 - output_q2_loss: 0.3967 - output_q3_loss: 0.3455 - output_q1_mean_squared_error: 28.5760 - output_q1_root_mean_squared_error: 5.3457 - output_q1_q0.05: 0.2145 - output_q1_q0.5: 1.9532 - output_q1_q0.95: 3.6918 - output_q2_mean_squared_error: 0.9573 - output_q2_root_me

...

6/6 [==============================] - 1s 152ms/step - loss: 0.5574 - output_q1_loss: 0.0830 - output_q2_loss: 0.3875 - output_q3_loss: 0.0869 - output_q1_mean_squared_error: 3.0163 - output_q1_root_mean_squared_error: 1.7368 - output_q1_q0.05: 0.0829 - output_q1_q0.5: 0.7208 - output_q1_q0.95: 1.3586 - output_q2_mean_squared_error: 0.9194 - output_q2_root_mean_squared_error: 0.9589 - output_q2_q0.05: 0.3842 - output_q2_q0.5: 0.3871 - output_q2_q0.95: 0.3901 - output_q3_mean_squared_error: 3.1844 - output_q3_root_mean_squared_error: 1.7845 - output_q3_q0.05: 1.4217 - output_q3_q0.5: 0.7543 - output_q3_q0.95: 0.0868
Epoch 242/300
6/6 [==============================] - 1s 143ms/step - loss: 0.5569 - output_q1_loss: 0.0829 - output_q2_loss: 0.3872 - output_q3_loss: 0.0869 - output_q1_mean_squared_error: 2.9736 - output_q1_root_mean_squared_error: 1.7244 - output_q1_q0.05: 0.0829 - output_q1_q0.5: 0.7135 - output_q1_q0.95: 1.3442 - output_q2_mean_squared_error: 0.9178 - output_q2_root_mean

6/6 [==============================] - 1s 147ms/step - loss: 0.5566 - output_q1_loss: 0.0828 - output_q2_loss: 0.3870 - output_q3_loss: 0.0868 - output_q1_mean_squared_error: 3.0228 - output_q1_root_mean_squared_error: 1.7386 - output_q1_q0.05: 0.0829 - output_q1_q0.5: 0.7214 - output_q1_q0.95: 1.3599 - output_q2_mean_squared_error: 0.9196 - output_q2_root_mean_squared_error: 0.9590 - output_q2_q0.05: 0.3811 - output_q2_q0.5: 0.3871 - output_q2_q0.95: 0.3931 - output_q3_mean_squared_error: 3.1473 - output_q3_root_mean_squared_error: 1.7741 - output_q3_q0.05: 1.4102 - output_q3_q0.5: 0.7485 - output_q3_q0.95: 0.0868
Epoch 254/300
6/6 [==============================] - 1s 127ms/step - loss: 0.5570 - output_q1_loss: 0.0826 - output_q2_loss: 0.3874 - output_q3_loss: 0.0870 - output_q1_mean_squared_error: 2.9848 - output_q1_root_mean_squared_error: 1.7277 - output_q1_q0.05: 0.0826 - output_q1_q0.5: 0.7143 - output_q1_q0.95: 1.3459 - output_q2_mean_squared_error: 0.9201 - output_q2_root_mean

6/6 [==============================] - 1s 131ms/step - loss: 0.5567 - output_q1_loss: 0.0828 - output_q2_loss: 0.3875 - output_q3_loss: 0.0865 - output_q1_mean_squared_error: 2.9945 - output_q1_root_mean_squared_error: 1.7305 - output_q1_q0.05: 0.0827 - output_q1_q0.5: 0.7171 - output_q1_q0.95: 1.3515 - output_q2_mean_squared_error: 0.9207 - output_q2_root_mean_squared_error: 0.9595 - output_q2_q0.05: 0.3853 - output_q2_q0.5: 0.3870 - output_q2_q0.95: 0.3888 - output_q3_mean_squared_error: 3.1430 - output_q3_root_mean_squared_error: 1.7728 - output_q3_q0.05: 1.4082 - output_q3_q0.5: 0.7473 - output_q3_q0.95: 0.0865
Epoch 266/300
6/6 [==============================] - 1s 144ms/step - loss: 0.5565 - output_q1_loss: 0.0827 - output_q2_loss: 0.3872 - output_q3_loss: 0.0866 - output_q1_mean_squared_error: 2.9617 - output_q1_root_mean_squared_error: 1.7210 - output_q1_q0.05: 0.0828 - output_q1_q0.5: 0.7119 - output_q1_q0.95: 1.3411 - output_q2_mean_squared_error: 0.9209 - output_q2_root_mean

6/6 [==============================] - 1s 148ms/step - loss: 0.5563 - output_q1_loss: 0.0827 - output_q2_loss: 0.3872 - output_q3_loss: 0.0865 - output_q1_mean_squared_error: 2.9636 - output_q1_root_mean_squared_error: 1.7215 - output_q1_q0.05: 0.0826 - output_q1_q0.5: 0.7097 - output_q1_q0.95: 1.3368 - output_q2_mean_squared_error: 0.9217 - output_q2_root_mean_squared_error: 0.9601 - output_q2_q0.05: 0.3764 - output_q2_q0.5: 0.3872 - output_q2_q0.95: 0.3981 - output_q3_mean_squared_error: 3.1339 - output_q3_root_mean_squared_error: 1.7703 - output_q3_q0.05: 1.4025 - output_q3_q0.5: 0.7445 - output_q3_q0.95: 0.0864
Epoch 278/300
6/6 [==============================] - 1s 158ms/step - loss: 0.5562 - output_q1_loss: 0.0828 - output_q2_loss: 0.3869 - output_q3_loss: 0.0864 - output_q1_mean_squared_error: 3.0707 - output_q1_root_mean_squared_error: 1.7523 - output_q1_q0.05: 0.0829 - output_q1_q0.5: 0.7291 - output_q1_q0.95: 1.3753 - output_q2_mean_squared_error: 0.9213 - output_q2_root_mean

6/6 [==============================] - 1s 149ms/step - loss: 0.5566 - output_q1_loss: 0.0829 - output_q2_loss: 0.3873 - output_q3_loss: 0.0864 - output_q1_mean_squared_error: 3.0477 - output_q1_root_mean_squared_error: 1.7458 - output_q1_q0.05: 0.0829 - output_q1_q0.5: 0.7241 - output_q1_q0.95: 1.3653 - output_q2_mean_squared_error: 0.9219 - output_q2_root_mean_squared_error: 0.9602 - output_q2_q0.05: 0.3885 - output_q2_q0.5: 0.3873 - output_q2_q0.95: 0.3860 - output_q3_mean_squared_error: 3.0749 - output_q3_root_mean_squared_error: 1.7535 - output_q3_q0.05: 1.3862 - output_q3_q0.5: 0.7363 - output_q3_q0.95: 0.0865
Epoch 290/300
6/6 [==============================] - 1s 146ms/step - loss: 0.5561 - output_q1_loss: 0.0826 - output_q2_loss: 0.3869 - output_q3_loss: 0.0867 - output_q1_mean_squared_error: 2.9830 - output_q1_root_mean_squared_error: 1.7271 - output_q1_q0.05: 0.0826 - output_q1_q0.5: 0.7143 - output_q1_q0.95: 1.3461 - output_q2_mean_squared_error: 0.9218 - output_q2_root_mean

Evaluate models.

In [ ]:
multi_out_lstm_eval = multi_out_lstm.evaluate([test_x, enc_names], [multi_test_y[0], multi_test_y[1], multi_test_y[2]],
                                              return_dict=True)

In [ ]:
multi_out_gru_eval = multi_out_gru.evaluate([test_x, enc_names], [multi_test_y[0], multi_test_y[1], multi_test_y[2]],
                                            return_dict=True)

In [ ]:
multi_out_lstm_V2_eval = multi_out_lstm_V2.evaluate([test_x, enc_names], [multi_test_y[0], multi_test_y[1], multi_test_y[2]],
                                                    return_dict=True)

In [ ]:
multi_out_gru_V2_eval = multi_out_gru_V2.evaluate([test_x, enc_names], [multi_test_y[0], multi_test_y[1], multi_test_y[2]],
                                                  return_dict=True)

In [ ]:
single_out_lstm_eval = single_out_lstm.evaluate([test_x, enc_names], test_y, return_dict=True)

In [ ]:
single_out_gru_eval = single_out_gru.evaluate([test_x, enc_names], test_y, return_dict=True)

In [29]:
multi_lstm_eval = multi_lstm_quant.evaluate([test_x, enc_names], [multi_train_y[0], multi_train_y[0], multi_train_y[0],
                                                                  multi_train_y[1], multi_train_y[1], multi_train_y[1],
                                                                  multi_train_y[2], multi_train_y[2], multi_train_y[2]],
                                            return_dict=True)

6/6 [==============================] - 0s 58ms/step - loss: 0.5837 - confirmed_q1_loss: 0.0298 - confirmed_q2_loss: 0.0937 - confirmed_q3_loss: 0.0286 - deceased_q1_loss: 0.0451 - deceased_q2_loss: 0.1369 - deceased_q3_loss: 0.0450 - recovered_q1_loss: 0.0396 - recovered_q2_loss: 0.1237 - recovered_q3_loss: 0.0412 - confirmed_q1_mean_squared_error: 0.3367 - confirmed_q1_root_mean_squared_error: 0.5802 - confirmed_q1_q0.05: 0.0299 - confirmed_q1_q0.5: 0.2309 - confirmed_q1_q0.95: 0.4319 - confirmed_q2_mean_squared_error: 0.0890 - confirmed_q2_root_mean_squared_error: 0.2984 - confirmed_q2_q0.05: 0.0939 - confirmed_q2_q0.5: 0.0938 - confirmed_q2_q0.95: 0.0937 - confirmed_q3_mean_squared_error: 0.2993 - confirmed_q3_root_mean_squared_error: 0.5471 - confirmed_q3_q0.05: 0.4040 - confirmed_q3_q0.5: 0.2163 - confirmed_q3_q0.95: 0.0285 - deceased_q1_mean_squared_error: 0.7582 - deceased_q1_root_mean_squared_error: 0.8707 - deceased_q1_q0.05: 0.0449 - deceased_q1_q0.5: 0.3513 - deceased_q1_q0.

In [30]:
multi_gru_eval = multi_gru_quant.evaluate([test_x, enc_names], [multi_train_y[0], multi_train_y[0], multi_train_y[0],
                                                                multi_train_y[1], multi_train_y[1], multi_train_y[1],
                                                                multi_train_y[2], multi_train_y[2], multi_train_y[2]],
                                          return_dict=True)

6/6 [==============================] - 0s 46ms/step - loss: 0.5661 - confirmed_q1_loss: 0.0308 - confirmed_q2_loss: 0.0886 - confirmed_q3_loss: 0.0294 - deceased_q1_loss: 0.0484 - deceased_q2_loss: 0.1250 - deceased_q3_loss: 0.0466 - recovered_q1_loss: 0.0388 - recovered_q2_loss: 0.1167 - recovered_q3_loss: 0.0418 - confirmed_q1_mean_squared_error: 0.2678 - confirmed_q1_root_mean_squared_error: 0.5175 - confirmed_q1_q0.05: 0.0307 - confirmed_q1_q0.5: 0.1853 - confirmed_q1_q0.95: 0.3399 - confirmed_q2_mean_squared_error: 0.0881 - confirmed_q2_root_mean_squared_error: 0.2967 - confirmed_q2_q0.05: 0.0886 - confirmed_q2_q0.5: 0.0885 - confirmed_q2_q0.95: 0.0884 - confirmed_q3_mean_squared_error: 0.2573 - confirmed_q3_root_mean_squared_error: 0.5073 - confirmed_q3_q0.05: 0.3413 - confirmed_q3_q0.5: 0.1853 - confirmed_q3_q0.95: 0.0294 - deceased_q1_mean_squared_error: 0.5633 - deceased_q1_root_mean_squared_error: 0.7505 - deceased_q1_q0.05: 0.0482 - deceased_q1_q0.5: 0.2901 - deceased_q1_q0.

In [26]:
single_lstm_eval = single_lstm_quant.evaluate([test_x, enc_names], [test_y, test_y, test_y], return_dict=True)

6/6 [==============================] - 0s 60ms/step - loss: 0.5564 - output_q1_loss: 0.0850 - output_q2_loss: 0.3832 - output_q3_loss: 0.0882 - output_q1_mean_squared_error: 2.8395 - output_q1_root_mean_squared_error: 1.6851 - output_q1_q0.05: 0.0850 - output_q1_q0.5: 0.6969 - output_q1_q0.95: 1.3089 - output_q2_mean_squared_error: 0.9031 - output_q2_root_mean_squared_error: 0.9503 - output_q2_q0.05: 0.3650 - output_q2_q0.5: 0.3832 - output_q2_q0.95: 0.4015 - output_q3_mean_squared_error: 3.1579 - output_q3_root_mean_squared_error: 1.7770 - output_q3_q0.05: 1.4222 - output_q3_q0.5: 0.7552 - output_q3_q0.95: 0.0882


In [27]:
single_gru_eval = single_gru_quant.evaluate([test_x, enc_names], [test_y, test_y, test_y], return_dict=True)

6/6 [==============================] - 0s 47ms/step - loss: 0.5595 - output_q1_loss: 0.0864 - output_q2_loss: 0.3833 - output_q3_loss: 0.0898 - output_q1_mean_squared_error: 2.8474 - output_q1_root_mean_squared_error: 1.6874 - output_q1_q0.05: 0.0863 - output_q1_q0.5: 0.6972 - output_q1_q0.95: 1.3081 - output_q2_mean_squared_error: 0.8995 - output_q2_root_mean_squared_error: 0.9484 - output_q2_q0.05: 0.3811 - output_q2_q0.5: 0.3834 - output_q2_q0.95: 0.3856 - output_q3_mean_squared_error: 2.9639 - output_q3_root_mean_squared_error: 1.7216 - output_q3_q0.05: 1.3577 - output_q3_q0.5: 0.7237 - output_q3_q0.95: 0.0897


Does multi output affect performance? In theory the models should be the same.<br>
Shared parameters use the TimeDistributed function while using the Dense size will create individual parameters for each day.<br>
To have the model output three quantiles use multi output with each output node optimized on a pinball loss with a different<br>
quantile value.

Callbacks: TerminateOnNAN<br>
Models (one LSTM and one GRU):<br>
Multi output individual. (OK)<br>
Multi output shared. (OK)<br>
Single output shared. (OK)<br>
Multi output quantile.

Test tanh on multi output LSTM models.<br>
Test regularization L1 and/or L2.<br>
Apply gradient clipping.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
def plot_training_history(hist, title):
    fig, ax = plt.subplots()
    ax.plot(hist.history['loss'])
    ax.plot(hist.history['confirmed_loss'])
    ax.plot(hist.history['deceased_loss'])
    ax.plot(hist.history['recovered_loss'])
    ax.set_title(f'model loss on train data: {title}')
    ax.set_ylabel('loss')
    ax.set_xlabel('epoch')
    ax.legend(['loss', 'confirmed_loss', 'deceased_loss', 'recovered_loss'], loc='best')

In [ ]:
def plot_training_history_single(hist, title):
    fig, ax = plt.subplots()
    ax.plot(hist.history['loss'])
    ax.set_title(f'model loss on train data: {title}')
    ax.set_ylabel('loss')
    ax.set_xlabel('epoch')
    ax.legend(['loss'], loc='best')

In [ ]:
def plot_training_metrics(hist, title):
    fig, ax = plt.subplots()
    ax.plot(hist.history['confirmed_mean_squared_error'])
    ax.plot(hist.history['confirmed_root_mean_squared_error'])
    ax.plot(hist.history['deceased_mean_squared_error'])
    ax.plot(hist.history['deceased_root_mean_squared_error'])
    ax.plot(hist.history['recovered_mean_squared_error'])
    ax.plot(hist.history['recovered_root_mean_squared_error'])
    ax.set_title(f'model error metrics on train data: {title}')
    ax.set_ylabel('error')
    ax.set_xlabel('epoch')
    ax.legend(['confirmed_mean_squared_error', 'confirmed_root_mean_squared_error', 'deceased_mean_squared_error', 'deceased_root_mean_squared_error', 'recovered_mean_squared_error', 'recovered_root_mean_squared_error'], loc='best')

In [ ]:
def plot_training_metrics_single(hist, title):
    fig, ax = plt.subplots()
    ax.plot(hist.history['root_mean_squared_error'])
    ax.plot(hist.history['mean_squared_error'])
    ax.set_title(f'model error metrics on train data: {title}')
    ax.set_ylabel('error')
    ax.set_xlabel('epoch')
    ax.legend(['root_mean_squared_error', 'mean_squared_error'], loc='best')

In [ ]:
plot_training_history(multi_out_lstm_hist, "multi_out_lstm_hist")
plot_training_history(multi_out_gru_hist, "multi_out_gru_hist")
plot_training_history(multi_out_lstm_V2_hist, "multi_out_lstm_V2_hist")
plot_training_history(multi_out_gru_V2_hist, "multi_out_gru_V2_hist")
plot_training_history_single(single_out_lstm_hist, "single_out_lstm_hist")
plot_training_history_single(single_out_gru_hist, "single_out_gru_hist")

In [ ]:
plot_training_metrics(multi_out_lstm_hist, "multi_out_lstm_hist")
plot_training_metrics(multi_out_gru_hist, "multi_out_gru_hist")
plot_training_metrics(multi_out_lstm_V2_hist, "multi_out_lstm_V2_hist")
plot_training_metrics(multi_out_gru_V2_hist, "multi_out_gru_V2_hist")
plot_training_metrics_single(single_out_lstm_hist, "single_out_lstm_hist")
plot_training_metrics_single(single_out_gru_hist, "single_out_gru_hist")

In [ ]:
from pprint import pprint
print("multi_out_lstm_eval")
pprint(multi_out_lstm_eval)
print()
print("multi_out_gru_eval")
pprint(multi_out_gru_eval)
print()
print("multi_out_lstm_V2_eval")
pprint(multi_out_lstm_V2_eval)
print()
print("multi_out_gru_V2_eval")
pprint(multi_out_gru_V2_eval)
print()
print("single_out_lstm_eval")
pprint(single_out_lstm_eval)
print()
print("single_out_gru_eval")
pprint(single_out_gru_eval)

In [32]:
from pprint import pprint
print("multi_lstm_eval")
pprint(multi_lstm_eval)
print()
print("multi_gru_eval")
pprint(multi_gru_eval)
print()
print("single_lstm_eval")
pprint(single_lstm_eval)
print()
print("single_gru_eval")
pprint(single_gru_eval)

multi_lstm_eval
{'confirmed_q1_loss': 0.029848609119653702,
 'confirmed_q1_mean_squared_error': 0.3366776704788208,
 'confirmed_q1_q0.05': 0.029875434935092926,
 'confirmed_q1_q0.5': 0.23088645935058594,
 'confirmed_q1_q0.95': 0.43189749121665955,
 'confirmed_q1_root_mean_squared_error': 0.5802392959594727,
 'confirmed_q2_loss': 0.09365778416395187,
 'confirmed_q2_mean_squared_error': 0.08904951065778732,
 'confirmed_q2_q0.05': 0.09387582540512085,
 'confirmed_q2_q0.5': 0.09379947930574417,
 'confirmed_q2_q0.95': 0.0937231257557869,
 'confirmed_q2_root_mean_squared_error': 0.29841166734695435,
 'confirmed_q3_loss': 0.02855551987886429,
 'confirmed_q3_mean_squared_error': 0.29932624101638794,
 'confirmed_q3_q0.05': 0.4040064811706543,
 'confirmed_q3_q0.5': 0.21626605093479156,
 'confirmed_q3_q0.95': 0.028525618836283684,
 'confirmed_q3_root_mean_squared_error': 0.5471071600914001,
 'deceased_q1_loss': 0.045093946158885956,
 'deceased_q1_mean_squared_error': 0.7581765055656433,
 'decease